In [1]:
%pip install "langchain[google-genai]"

  Using cached google_ai_generativelanguage-0.6.18-py3-none-any.whl.metadata (9.8 kB)
Using cached google_ai_generativelanguage-0.6.18-py3-none-any.whl (1.4 MB)
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


In [2]:
# !pip install langchain-huggingface
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [3]:
from dotenv import load_dotenv
load_dotenv(r"D:\py_prac\langchain-prac\.env")

True

In [4]:
import getpass
import os

if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

from langchain.chat_models import init_chat_model

model = init_chat_model("gemini-2.0-flash", model_provider="google_genai")

In [5]:
model_resp = model.invoke("who are you, which model")
model_resp.to_json()

{'lc': 1,
 'type': 'constructor',
 'id': ['langchain', 'schema', 'messages', 'AIMessage'],
 'kwargs': {'content': 'I am a large language model, trained by Google.',
  'response_metadata': {'prompt_feedback': {'block_reason': 0,
    'safety_ratings': []},
   'finish_reason': 'STOP',
   'model_name': 'gemini-2.0-flash',
   'safety_ratings': []},
  'type': 'ai',
  'id': 'run--c3b0dfa6-4a98-4f96-b510-b7239f71756d-0',
  'usage_metadata': {'input_tokens': 6,
   'output_tokens': 12,
   'total_tokens': 18,
   'input_token_details': {'cache_read': 0}},
  'tool_calls': [],
  'invalid_tool_calls': []}}

In [6]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage("Translate the following from English into hindi. also add an example of the translation in a sentence. Do not use any other language in the response. Use the format: 'Translation: <translated text>. Example: <example sentence in hindi>'."),
    HumanMessage("hi!"),
]

resp = model.invoke(messages)
resp.content

'अनुवाद: नमस्ते!\nउदाहरण: नमस्ते, आप कैसे हैं?'

In [7]:
type(resp)

langchain_core.messages.ai.AIMessage

In [8]:
dir(resp)

['__abstractmethods__',
 '__add__',
 '__annotations__',
 '__class__',
 '__class_getitem__',
 '__class_vars__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__fields__',
 '__fields_set__',
 '__format__',
 '__ge__',
 '__get_pydantic_core_schema__',
 '__get_pydantic_json_schema__',
 '__getattr__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__pretty__',
 '__private_attributes__',
 '__pydantic_complete__',
 '__pydantic_computed_fields__',
 '__pydantic_core_schema__',
 '__pydantic_custom_init__',
 '__pydantic_decorators__',
 '__pydantic_extra__',
 '__pydantic_fields__',
 '__pydantic_fields_set__',
 '__pydantic_generic_metadata__',
 '__pydantic_init_subclass__',
 '__pydantic_parent_namespace__',
 '__pydantic_post_init__',
 '__pydantic_private__',
 '__pydantic_root_model__',
 '__pydantic_serializer__',
 '__pydantic_s

In [9]:
resp.pretty_print()

================================== Ai Message ==================================

अनुवाद: नमस्ते!
उदाहरण: नमस्ते, आप कैसे हैं?


In [10]:
for token in model.stream(messages):
    print(token.content, end="<..>")

अनु<..>वाद: नमस्ते!
उदाहरण: नमस्ते! आप कैसे हैं?
<..>

Streaming with async

In [11]:
from langchain_google_genai import ChatGoogleGenerativeAI

# Initialize the chat model
chatmodel = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

# Use async directly at the top level in Jupyter, ignore the error or just wrap the api call in the async function and just await that function (without a new async loop)
async for chunk in chatmodel.astream("Write me a 1 verse song about goldfish on the moon, of 150 words."):
    print(chunk.content, end="... ", flush=True)

(... Verse 1... )
In a bowl of moondust, a goldfish does reside,
Swimming...  through craters, with nowhere to hide.
He flicks his orange tail, in...  zero gravity's sway,
Dreaming of kelp forests, so very far away.
He stares back at Earth, a blue and swirling gem,
A...  forgotten memory, a watery diadem.
No bubbles to burst, no surface to reach,
Just lunar landscapes, beyond mortal speech.
He nibbles on moon...  rocks, a cosmic cuisine,
A solitary explorer, in a silver screen.
He wonders if anyone, down on the ground,
Sees his tiny spaceship, where he's forever bound.
A goldfish on the moon, a whimsical...  plight,
A shimmering beacon, in the endless night.
... 

OR

In [12]:
import nest_asyncio
import asyncio
from langchain_google_genai import ChatGoogleGenerativeAI

# nest_asyncio.apply()  # Patch the running event loop

async def main():
    chatmodel = ChatGoogleGenerativeAI(model="gemini-2.0-flash")
    async for chunk in chatmodel.astream("Write me a 1 verse song about goldfish on the moon"):
        print(chunk.content, end="|", flush=True)

# Works now in Colab/Jupyter
await main()


In| a silver bowl of stardust bright,
Swim goldfish, orange in the lunar| light.
No gravity holds, they float and they gleam,
A| silent ballet, a moonlit dream.
|

ChatPromptTemplate

In [13]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "Translate the following from English into {language}"

prompt_template = ChatPromptTemplate(
    [("system", system_template), ("user", "{text}")]
)
# prompt_template = ChatPromptTemplate.from_messages(
#     [("system", system_template), ("user", "{text}")]
# )
prompt = prompt_template.invoke({"language": "hindi", "text": "hi!"})
prompt

ChatPromptValue(messages=[SystemMessage(content='Translate the following from English into hindi', additional_kwargs={}, response_metadata={}), HumanMessage(content='hi!', additional_kwargs={}, response_metadata={})])

In [14]:
dir(prompt)

['__abstractmethods__',
 '__annotations__',
 '__class__',
 '__class_getitem__',
 '__class_vars__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__fields__',
 '__fields_set__',
 '__format__',
 '__ge__',
 '__get_pydantic_core_schema__',
 '__get_pydantic_json_schema__',
 '__getattr__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__pretty__',
 '__private_attributes__',
 '__pydantic_complete__',
 '__pydantic_computed_fields__',
 '__pydantic_core_schema__',
 '__pydantic_custom_init__',
 '__pydantic_decorators__',
 '__pydantic_extra__',
 '__pydantic_fields__',
 '__pydantic_fields_set__',
 '__pydantic_generic_metadata__',
 '__pydantic_init_subclass__',
 '__pydantic_parent_namespace__',
 '__pydantic_post_init__',
 '__pydantic_private__',
 '__pydantic_root_model__',
 '__pydantic_serializer__',
 '__pydantic_setattr_handl

In [15]:
prompt.to_messages()

[SystemMessage(content='Translate the following from English into hindi', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='hi!', additional_kwargs={}, response_metadata={})]

In [16]:
response = model.invoke(prompt)
print(response.content)

नमस्ते! (Namaste!)

You can also say:

*   हेलो! (Hello!)
*   हाय! (Hi!) - This is a direct transliteration of "Hi" and is commonly used, especially among younger people.


In [17]:
chain = prompt_template | model
response = chain.invoke({"language": "sanskrit", "text": "hi!"})
print(response.content)

The most common and direct translation of "hi!" into Sanskrit would be:

**नमस्ते (Namaste)**

This is a standard greeting that conveys respect and goodwill.

While less common, you could also use:

*   **शुभम् (Shubham)** - This means "auspicious" or "well-being," and can be used as a greeting, wishing someone well.
*   **स्वस्ति (Svasti)** - This means "well-being" or "good fortune."
*   **अहो (Aho)** - This is similar to "O!" or "Hey!" and is more of an interjection.

However, **नमस्ते (Namaste)** is the most widely recognized and appropriate translation for "hi!" in most situations.


In [18]:
%pip install google-generativeai --upgrade


  Using cached google_ai_generativelanguage-0.6.15-py3-none-any.whl.metadata (5.7 kB)
Using cached google_ai_generativelanguage-0.6.15-py3-none-any.whl (1.3 MB)
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.18
    Uninstalling google-ai-generativelanguage-0.6.18:
      Successfully uninstalled google-ai-generativelanguage-0.6.18
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-google-genai 2.1.7 requires google-ai-generativelanguage<0.7.0,>=0.6.18, but you have google-ai-generativelanguage 0.6.15 which is incompatible.


In [19]:
from google import genai
from google.genai import types
import time
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage

client = genai.Client()

# Upload file
file = client.files.upload(file=r"D:\py_prac\langchain-prac\datasets\CPlusPlusNotesForProfessionals.pdf")
while file.state.name == 'PROCESSING':
    print(f"File {file.name} is still processing...")
    # Wait for processing to complete
    time.sleep(2)
    file = client.files.get(name=file.name)

# Create cache
model = 'gemini-2.0-flash'
cache = client.caches.create(
    model=model,
    config=types.CreateCachedContentConfig(
        display_name='Cached Content',
        system_instruction=(
            "You are an expert content analyzer, and your job is to answer the user's query based on the file you have access to."
        ),
        contents=[file],
        ttl="3000s",
    )
)

# Query with LangChain
llm = ChatGoogleGenerativeAI(
    model=model,
    cached_content=cache.name,
)
message = HumanMessage(content="Summarize the main points of the content.")
airesponse = llm.invoke([message])

In [20]:
airesponse.content

'The document is a comprehensive guide to C++ programming, covering a wide array of topics from basic syntax and data types to advanced concepts like metaprogramming, memory management, and concurrency. Here\'s a summary organized by key areas:\n\n*   **Fundamentals:** Covers the basics of C++, including program structure, comments, compilation, functions, and variable declarations.\n\n*   **Core Language Features:** Explores literals, operator precedence, bit operators, bit manipulation, arrays, iterators, and basic input/output.\n\n*   **Flow Control:** Discusses loops (for, while, do-while), loop control statements (break, continue), and conditional structures (if, else).\n\n*   **File I/O:** Details how to read from and write to files using C++ streams.\n\n*   **C++ Streams and Manipulators:** Explains the use of C++ streams for string manipulation and printing collections, as well as stream manipulators for formatting output.\n\n*   **Polymorphism and Inheritance:** Covers polymor

In [21]:
llm.invoke([HumanMessage(content="Tell me about the python related part in this pdf")]).content

'There is no mention of Python in the document.'

In [22]:
cache

CachedContent(name='cachedContents/tose3ga85k1mfgvqlxqh92ncuxu546o7o8cvi8qx', display_name='Cached Content', model='models/gemini-2.0-flash', create_time=datetime.datetime(2025, 7, 12, 12, 41, 7, 300660, tzinfo=TzInfo(UTC)), update_time=datetime.datetime(2025, 7, 12, 12, 41, 7, 300660, tzinfo=TzInfo(UTC)), expire_time=datetime.datetime(2025, 7, 12, 13, 31, 6, 405315, tzinfo=TzInfo(UTC)), usage_metadata=CachedContentUsageMetadata(audio_duration_seconds=None, image_count=None, text_count=None, total_token_count=182693, video_duration_seconds=None))

In [23]:
dir(client)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_aio',
 '_api_client',
 '_batches',
 '_caches',
 '_debug_config',
 '_files',
 '_get_api_client',
 '_models',
 '_operations',
 '_tunings',
 'aio',
 'batches',
 'caches',
 'chats',
 'files',
 'models',
 'operations',
 'tunings',
 'vertexai']

In [24]:
client.caches

In [25]:
file.name

'files/wp8hxo7y4hki'

In [26]:

# client.caches.get(name="files/rvmpfysdfu0m")

In [27]:
file.state.name

'ACTIVE'

In [28]:
type(airesponse)

langchain_core.messages.ai.AIMessage

In [29]:
airesponse.content

'The document is a comprehensive guide to C++ programming, covering a wide array of topics from basic syntax and data types to advanced concepts like metaprogramming, memory management, and concurrency. Here\'s a summary organized by key areas:\n\n*   **Fundamentals:** Covers the basics of C++, including program structure, comments, compilation, functions, and variable declarations.\n\n*   **Core Language Features:** Explores literals, operator precedence, bit operators, bit manipulation, arrays, iterators, and basic input/output.\n\n*   **Flow Control:** Discusses loops (for, while, do-while), loop control statements (break, continue), and conditional structures (if, else).\n\n*   **File I/O:** Details how to read from and write to files using C++ streams.\n\n*   **C++ Streams and Manipulators:** Explains the use of C++ streams for string manipulation and printing collections, as well as stream manipulators for formatting output.\n\n*   **Polymorphism and Inheritance:** Covers polymor

## Agent using langgraph

In [30]:
%pip install -U langgraph langchain-tavily langgraph-checkpoint-sqlite

Note: you may need to restart the kernel to use updated packages.


In [31]:
from langchain_tavily import TavilySearch
from langchain.tools import tool
from langchain_core.messages import HumanMessage

In [32]:
import os
if not os.environ.get("TAVILY_API_KEY"):
  os.environ["TAVILY_API_KEY"] = getpass.getpass("Enter API key for TAVILY_API_KEY: ")

In [33]:
search = TavilySearch(max_results=2)
search_results = search.invoke("What is the weather in New Delhi")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
# from pydantic import BaseModel

# class SearchInput(BaseModel):
#     query: str

@tool
def searchtool(query) -> str:
    """Search the web for recent information about anything you don't know."""
    print(f"In the search tool now: params i got - > query:{query}")
    result = search.invoke(query)
    return str(result)

tools = [searchtool]

{'query': 'What is the weather in New Delhi', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'Weather in New Delhi', 'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'New Delhi', 'region': 'Delhi', 'country': 'India', 'lat': 28.6, 'lon': 77.2, 'tz_id': 'Asia/Kolkata', 'localtime_epoch': 1752324285, 'localtime': '2025-07-12 18:14'}, 'current': {'last_updated_epoch': 1752323400, 'last_updated': '2025-07-12 18:00', 'temp_c': 37.5, 'temp_f': 99.5, 'is_day': 1, 'condition': {'text': 'Partly Cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 3.8, 'wind_kph': 6.1, 'wind_degree': 177, 'wind_dir': 'S', 'pressure_mb': 998.0, 'pressure_in': 29.48, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 35, 'cloud': 30, 'feelslike_c': 41.0, 'feelslike_f': 105.8, 'windchill_c': 37.5, 'windchill_f': 99.5, 'heatindex_c': 41.0, 'heatindex_f': 105.8, 'dewpoint_c': 19.3, 'dewpoint_f': 66.7, 'vis_km': 10.0, 'vis_m

In [34]:
import getpass
import os

if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

from langchain.chat_models import init_chat_model

model = init_chat_model("gemini-2.0-flash", model_provider="google_genai")

In [35]:
query = "Hi! i am V"
# response = model.invoke([{"role": "user", "content": query}])
# OR 
response = model.invoke([HumanMessage(content=query)])
response.text()

"Hi V! It's nice to meet you. How can I help you today?"

In [36]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage, ToolMessage

model_plus_tools = model.bind_tools(tools)

query = "What is the weather in New Delhi?"
# response = model_plus_tools.invoke([{"role": "user", "content": query}])
response = model_plus_tools.invoke([HumanMessage(content=query)])

print(f"Message content: {response.text()}\n")
# here there is no response but as we can see that the tool is being MARKED TO BE called.
print(f"Tool calls: {response.tool_calls}")

Message content: 

Tool calls: [{'name': 'searchtool', 'args': {'query': 'weather in New Delhi'}, 'id': '3e7a6e6d-70f9-4e57-8a56-93dd83f47ffc', 'type': 'tool_call'}]


In [37]:
response.tool_calls[0]

{'name': 'searchtool',
 'args': {'query': 'weather in New Delhi'},
 'id': '3e7a6e6d-70f9-4e57-8a56-93dd83f47ffc',
 'type': 'tool_call'}

In [38]:
# Now to actually call the tool, we need to use the tool calls in the response.
# Now we can call the tool with the tool call.
tool_call = response.tool_calls[0]
print(tool_call)
print("args = ", tool_call["args"])
tool_response = search.invoke(tool_call["args"])
tool_response_content = tool_response["results"][0]
final_response = model.invoke([AIMessage(content=str(tool_response_content)), HumanMessage(content="write this response in hinglish, pointwise, and in a friendly tone")])
print(f"\n\n ------------------ {final_response.content}")

{'name': 'searchtool', 'args': {'query': 'weather in New Delhi'}, 'id': '3e7a6e6d-70f9-4e57-8a56-93dd83f47ffc', 'type': 'tool_call'}
args =  {'query': 'weather in New Delhi'}


 ------------------ Okay, dost! Here's the weather report for New Delhi, straight from the source, but in Hinglish and easy to understand:

*   **Location:** New Delhi, Delhi, India. Apni Dilli!
*   **Time:** Abhi baj rahe hain 6:14 PM (shaam ke), 12th July, 2025.
*   **Temperature:** 37.5 degrees Celsius (garmi hai bhai!), which is like 99.5 degrees Fahrenheit.
*   **Condition:** Partly Cloudy. Thode badal hai, thodi dhoop.
*   **Wind:** Hawa chal rahi hai 6.1 kilometers per hour se.
*   **Humidity:** 35%. Itni bhi zyada nami nahi hai.
*   **Feels Like:** Lagbhag 41 degrees Celsius (105.8 Fahrenheit). Toh matlab, asal temperature se zyada garam lagega!
*   **Important Note:** Heat index bhi 41 degrees Celsius hai. Toh dhyan rakhna, dehydrate mat hona. Pani peete rehna!
*   **UV Index:** UV index 0.1 hai.

So, b

### Create the agent using LangGraph now (using the above same tools)

In [39]:
from langgraph.prebuilt import create_react_agent

# Note that we are passing in the model, not model_with_tools. That is because create_react_agent will call .bind_tools for us under the hood.
react_agent_executor = create_react_agent(
    model=model,
    tools=tools,
    prompt="You are a helpful assistant. Use tools when needed."
)

In [40]:
# Now using a relevant input prompt
input_message = {"role": "user", "content": "What is the current weather in New Delhi?"}
response = react_agent_executor.invoke({"messages": [input_message]})

for message in response["messages"]:
    message.pretty_print()

response

In the search tool now: params i got - > query:weather in New Delhi
================================ Human Message =================================

What is the current weather in New Delhi?
================================== Ai Message ==================================
Tool Calls:
  searchtool (db150aac-fc4e-4f00-81b2-4d0254e750d2)
 Call ID: db150aac-fc4e-4f00-81b2-4d0254e750d2
  Args:
    query: weather in New Delhi
================================= Tool Message =================================
Name: searchtool

{'query': 'weather in New Delhi', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'Weather in New Delhi', 'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'New Delhi', 'region': 'Delhi', 'country': 'India', 'lat': 28.6, 'lon': 77.2, 'tz_id': 'Asia/Kolkata', 'localtime_epoch': 1752324285, 'localtime': '2025-07-12 18:14'}, 'current': {'last_updated_epoch': 1752323400, 'last_updated': '2025-07-12 18:00', 'temp_c': 37.5, 

{'messages': [HumanMessage(content='What is the current weather in New Delhi?', additional_kwargs={}, response_metadata={}, id='d4fffe19-741b-4b4c-829d-995cf0a483f9'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'searchtool', 'arguments': '{"query": "weather in New Delhi"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--9a7c9a14-e934-4b04-a89f-a5286285c061-0', tool_calls=[{'name': 'searchtool', 'args': {'query': 'weather in New Delhi'}, 'id': 'db150aac-fc4e-4f00-81b2-4d0254e750d2', 'type': 'tool_call'}], usage_metadata={'input_tokens': 39, 'output_tokens': 7, 'total_tokens': 46, 'input_token_details': {'cache_read': 0}}),
  ToolMessage(content='{\'query\': \'weather in New Delhi\', \'follow_up_questions\': None, \'answer\': None, \'images\': [], \'results\': [{\'title\': \'Weather in New Delhi\', \'url\': \'https://www.weatherapi.com/\

In [41]:
# WORKING CODE !!!

# 1️⃣ Imports & setup
import os, getpass
from langchain.chat_models import init_chat_model
from langchain_tavily import TavilySearch
from langchain_core.tools import tool
from langgraph.prebuilt import create_react_agent

# 2️⃣ API keys
if not os.environ.get("GOOGLE_API_KEY"):
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Google Gemini API key: ")
if not os.environ.get("TAVILY_API_KEY"):
    os.environ["TAVILY_API_KEY"] = getpass.getpass("Tavily API key: ")

# 3️⃣ Model initialization
model = init_chat_model(
    "gemini-2.0-flash",
    model_provider="google_genai"
)

# 4️⃣ TavilySearch instance
search_instance = TavilySearch(max_results=2)

# 5️⃣ Tool definition (match `query` signature)
@tool
def search(query: str) -> str:
    """Search the web for `query`."""
    result = search_instance.invoke(query)
    return str(result)

tools = [search]

# 6️⃣ Create ReAct agent (auto .bind_tools under the hood)
react_agent = create_react_agent(model=model, tools=tools)

# 7️⃣ Invoke and print
resp = react_agent.invoke({
    "messages": [{"role": "user", "content": "What is the current weather in New Delhi?"}]
})
print(resp["messages"][-1].content)


The weather in New Delhi is partly cloudy with a temperature of 37.5°C (99.5°F). It feels like 41.0°C (105.8°F). There is a 35% humidity and a light wind of 6.1 km/h from the south. There is a chance of patchy rain in the evening.


In [42]:
# Import standard libraries
import os
import getpass

# Import the LangChain components
from langchain.chat_models import init_chat_model
from langchain_tavily import TavilySearch
from langchain_core.tools import tool

# Import Pydantic for structured inputs
from pydantic import BaseModel

# Import ReAct agent creation
from langgraph.prebuilt import create_react_agent


In [43]:
## Tool 1: search_web

# Pydantic schema for search_web inputs
class WebSearchInput(BaseModel):
    query: str
    num_results: int

# Tool definition using Pydantic
@tool
def search_web(input_data: WebSearchInput) -> str:
    """
    Search the web with a query and number of results.
    """
    # Perform search
    result = search_instance.invoke(input_data.query, k=input_data.num_results)
    # Return stringified result
    return str(result)


In [44]:
## Tool 2: search_weather

# Pydantic schema for weather search inputs
class WeatherSearchInput(BaseModel):
    city: str
    unit: str  # e.g., "Celsius" or "Fahrenheit"

@tool
def search_weather(input_data: WeatherSearchInput) -> str:
    """
    Search weather information for a city.
    """
    query = f"current weather in {input_data.city} in {input_data.unit}"
    result = search_instance.invoke(query)
    return str(result)


In [45]:
## Tool 3 : search_news

# Pydantic schema for news search inputs
class NewsSearchInput(BaseModel):
    topic: str
    region: str

@tool
def search_news(input_data: NewsSearchInput) -> str:
    """
    Search recent news headlines for a topic in a specific region.
    """
    query = f"latest news about {input_data.topic} in {input_data.region}"
    result = search_instance.invoke(query)
    return str(result)


In [46]:
## Register Tools
# All tools are collected here
tools = [
    search_web,
    search_weather,
    search_news
]


In [47]:
# Create ReAct agent with the model and tools
react_agent_2 = create_react_agent(
    model=model,
    tools=tools
)


In [48]:
from pprint import pprint

In [49]:
# User query to trigger one of the tools
resp = react_agent_2.invoke({
    "messages": [
        {"role": "user", "content": "Give me the latest news and weather in India, new delhi"}
    ]
}
  )
pprint(len(resp["messages"]))

print(resp["messages"][-1].content)
# pprint(resp["messages"][-2].content)


for message in resp["messages"]:
    message.pretty_print()

5
Latest news in India: According to the Indian Express, Mann is stepping up his attack on PM Modi after MEA rap. According to the Times of India, breaking news includes sports, business, entertainment, blogs and opinions from leading columnists.

Weather in New Delhi: The weather is partly cloudy with a temperature of 37.5 degrees Celsius, feels like 41.0 degrees Celsius. The wind is blowing from the South at 6.1 km/h.
================================ Human Message =================================

Give me the latest news and weather in India, new delhi
================================== Ai Message ==================================
Tool Calls:
  search_news (a028b2ff-24de-4f27-ac11-9c0fdb0b1c66)
 Call ID: a028b2ff-24de-4f27-ac11-9c0fdb0b1c66
  Args:
    input_data: {'region': 'India', 'topic': 'latest'}
  search_weather (4ca0fa2c-d18a-4dec-8d86-815251467284)
 Call ID: 4ca0fa2c-d18a-4dec-8d86-815251467284
  Args:
    input_data: {'city': 'new delhi', 'unit': 'celsius'}
==============

In [50]:
# Import standard libraries
import os
import getpass
from typing import TypedDict
from pprint import pprint

# Import the LangChain components
from langchain.chat_models import init_chat_model
from langchain_tavily import TavilySearch
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage

# Import Pydantic for structured inputs
from pydantic import BaseModel

# Import ReAct agent creation
from langgraph.prebuilt import create_react_agent
from langgraph.graph import StateGraph, START, END

# API keys setup
if not os.environ.get("GOOGLE_API_KEY"):
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Google Gemini API key: ")
if not os.environ.get("TAVILY_API_KEY"):
    os.environ["TAVILY_API_KEY"] = getpass.getpass("Tavily API key: ")

# Model initialization
model = init_chat_model(
    "gemini-2.0-flash",
    model_provider="google_genai"
)

# TavilySearch instance
search_instance = TavilySearch(max_results=2)

## Tool 1: search_web
# Pydantic schema for search_web inputs
class WebSearchInput(BaseModel):
    query: str
    num_results: int = 2  # Default value

# Tool definition using Pydantic
@tool
def search_web(input_data: WebSearchInput) -> str:
    """
    Search the web with a query and number of results.
    """
    # Note: TavilySearch uses max_results parameter, not k
    search_temp = TavilySearch(max_results=input_data.num_results)
    result = search_temp.invoke(input_data.query)
    return str(result)

## Tool 2: search_weather
# Pydantic schema for weather search inputs
class WeatherSearchInput(BaseModel):
    city: str
    unit: str = "Celsius"  # Default value

@tool
def search_weather(input_data: WeatherSearchInput) -> str:
    """
    Search weather information for a city.
    """
    query = f"current weather in {input_data.city} in {input_data.unit}"
    result = search_instance.invoke(query)
    return str(result)

## Tool 3: search_news
# Pydantic schema for news search inputs
class NewsSearchInput(BaseModel):
    topic: str
    region: str

@tool
def search_news(input_data: NewsSearchInput) -> str:
    """
    Search recent news headlines for a topic in a specific region.
    """
    query = f"latest news about {input_data.topic} in {input_data.region}"
    result = search_instance.invoke(query)
    return str(result)

## Register Tools
# All tools are collected here
tools = [
    search_web,
    search_weather,
    search_news
]

# Create ReAct agent with the model and tools
react_agent = create_react_agent(
    model=model,
    tools=tools
)

# Test the basic ReAct agent
print("=== Testing Basic ReAct Agent ===")
resp = react_agent.invoke({
    "messages": [
        {"role": "user", "content": "Give me the latest news and weather in India, New Delhi"}
    ]
})

print(f"Number of messages: {len(resp['messages'])}")
print(f"Final response: {resp['messages'][-1].content}")



=== Testing Basic ReAct Agent ===
Number of messages: 5
Final response: Latest News: According to Indian Express, Mann is stepping up attack on PM Modi after MEA rap. Also, there is a discussion on why Shubman Gill wasn't asked to leave field for treatment.
Weather in New Delhi: The current weather in New Delhi is 37.5 degrees Celsius with partly cloudy conditions. The wind is blowing from the South at 6.1 km/h. It feels like 41.0 degrees Celsius.


In [51]:
# ✅ Define State for Custom Agent using TypedDict
class AgentState(TypedDict):
    user_input: str
    response: str

# ✅ Wrap ReAct agent as node
def run_react_agent(state: AgentState) -> AgentState:
    """Run the ReAct agent and update state with response"""
    # ReAct agent takes list of messages
    result = react_agent.invoke({
        "messages": [HumanMessage(content=state["user_input"])]
    })

    # Update state with response
    state["response"] = result["messages"][-1].content
    return state

# ✅ Create StateGraph
def create_custom_agent():
    """Create a custom agent using StateGraph"""

    # Create the graph
    workflow = StateGraph(AgentState)

    # Add the ReAct agent node
    workflow.add_node("react_agent", run_react_agent)

    # Define the flow
    workflow.add_edge(START, "react_agent")
    workflow.add_edge("react_agent", END)

    # Compile the graph
    return workflow.compile()

# ✅ Test the custom StateGraph agent
print("\n=== Testing Custom StateGraph Agent ===")
custom_agent = create_custom_agent()

# Test with initial state
initial_state = {
    "user_input": "What's the weather like in New Delhi today?",
    "response": ""
}

result = custom_agent.invoke(initial_state)
print(f"User Input: {result['user_input']}")
print(f"Agent Response: {result['response']}")

# ✅ Another test with news query
print("\n=== Testing News Query ===")
news_state = {
    "user_input": "Give me the latest technology news in India",
    "response": ""
}

news_result = custom_agent.invoke(news_state)
print(f"User Input: {news_result['user_input']}")
print(f"Agent Response: {news_result['response']}")


=== Testing Custom StateGraph Agent ===
User Input: What's the weather like in New Delhi today?
Agent Response: The weather in New Delhi is partly cloudy with a temperature of 37.5 degrees Celsius, but it feels like 41.0 degrees Celsius.

=== Testing News Query ===
User Input: Give me the latest technology news in India
Agent Response: OK. Here's the latest technology news in India:

*   Times of India is providing trending tech news, mobile phones, laptops, reviews, software updates.
*   The Hindu reports that Google Gemini has a new feature that can turn photos into video clips. Also, TikTok faces a fresh European privacy investigation over China data transfers.


### Complex langgraph workflow

In [52]:
# Import standard libraries
import os
import getpass
from typing import TypedDict, List, Optional, Literal
from pprint import pprint
import json
from datetime import datetime

# Import the LangChain components
from langchain.chat_models import init_chat_model
from langchain_tavily import TavilySearch
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage

# Import Pydantic for structured inputs
from pydantic import BaseModel

# Import ReAct agent creation
from langgraph.prebuilt import create_react_agent
from langgraph.graph import StateGraph, START, END

# API keys setup
if not os.environ.get("GOOGLE_API_KEY"):
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Google Gemini API key: ")
if not os.environ.get("TAVILY_API_KEY"):
    os.environ["TAVILY_API_KEY"] = getpass.getpass("Tavily API key: ")

# Model initialization
model = init_chat_model(
    "gemini-2.0-flash",
    model_provider="google_genai"
)

# TavilySearch instance
search_instance = TavilySearch(max_results=3)

# ===== PYDANTIC SCHEMAS =====
class WebSearchInput(BaseModel):
    query: str
    num_results: int = 2

class WeatherSearchInput(BaseModel):
    city: str
    unit: str = "Celsius"

class NewsSearchInput(BaseModel):
    topic: str
    region: str

# ===== TOOLS =====
@tool
def search_web(query: str, num_results: int = 2) -> str:
    """Search the web with a query and number of results."""
    search_temp = TavilySearch(max_results=num_results)
    result = search_temp.invoke(query)
    return str(result)

@tool
def search_weather(usrdata:WeatherSearchInput) -> str:
    """Search weather information for a city."""
    print(f"Received user data for weather search: {usrdata} of type {type(usrdata)}")
    # Use the Pydantic model to access city and unit
    query = f"current weather in {usrdata.city} in {usrdata.unit}"
    result = search_instance.invoke(query)
    return str(result)
# def search_weather(city: str, unit: str = "Celsius") -> str:
#     """Search weather information for a city."""
#     query = f"current weather in {city} in {unit}"
#     result = search_instance.invoke(query)
#     return str(result)

@tool
def search_news(topic: str, region: str = "India") -> str:
    """Search recent news headlines for a topic in a specific region."""
    query = f"latest news about {topic} in {region}"
    result = search_instance.invoke(query)
    return str(result)

# ===== COMPLEX STATE DEFINITION =====
class ComplexAgentState(TypedDict):
    # Input
    user_input: str

    # Processing stages
    intent: Optional[str]
    entities: Optional[dict]

    # Results from different nodes
    weather_data: Optional[str]
    news_data: Optional[str]
    general_search_data: Optional[str]

    # Error handling
    errors: List[str]
    retry_count: int

    # Node execution tracking
    execution_log: List[dict]

    # Final output
    final_response: str

    # Control flow
    next_action: Optional[str]

# ===== UTILITY FUNCTIONS =====
def log_execution(state: ComplexAgentState, node_name: str, status: str, details: str = ""):
    """Log node execution for debugging"""
    timestamp = datetime.now().isoformat()
    log_entry = {
        "timestamp": timestamp,
        "node": node_name,
        "status": status,
        "details": details
    }

    if "execution_log" not in state:
        state["execution_log"] = []

    state["execution_log"].append(log_entry)

    # Pretty print for real-time monitoring
    print(f"\n🔄 [{timestamp}] NODE: {node_name} | STATUS: {status}")
    if details:
        print(f"   📝 Details: {details}")

def pretty_print_state(state: ComplexAgentState, title: str):
    """Pretty print the current state"""
    print(f"\n{'='*50}")
    print(f"📊 {title}")
    print(f"{'='*50}")

    for key, value in state.items():
        if key == "execution_log":
            continue  # Skip execution log in state dump

        if isinstance(value, str) and len(value) > 100:
            print(f"{key}: {value[:100]}...")
        else:
            print(f"{key}: {value}")

    print(f"{'='*50}")

# ===== NODE FUNCTIONS =====

def intent_classifier(state: ComplexAgentState) -> ComplexAgentState:
    """Classify user intent and extract entities"""
    log_execution(state, "intent_classifier", "STARTED")

    try:
        user_input = state["user_input"].lower()

        # Simple intent classification
        if "weather" in user_input:
            state["intent"] = "weather"
        elif "news" in user_input:
            state["intent"] = "news"
        elif any(word in user_input for word in ["search", "find", "look"]):
            state["intent"] = "search"
        else:
            state["intent"] = "general"

        # Extract entities (simple keyword extraction)
        entities = {}

        # Extract city names (simple approach)
        cities = ["delhi", "mumbai", "bangalore", "chennai", "kolkata", "hyderabad"]
        for city in cities:
            if city in user_input:
                entities["city"] = city.title()
                break

        # Extract topics
        if "technology" in user_input or "tech" in user_input:
            entities["topic"] = "technology"
        elif "sports" in user_input:
            entities["topic"] = "sports"
        elif "politics" in user_input:
            entities["topic"] = "politics"

        state["entities"] = entities

        log_execution(state, "intent_classifier", "SUCCESS",
                     f"Intent: {state['intent']}, Entities: {entities}")

        return state

    except Exception as e:
        state["errors"].append(f"Intent classification failed: {str(e)}")
        log_execution(state, "intent_classifier", "ERROR", str(e))
        return state

def weather_node(state: ComplexAgentState) -> ComplexAgentState:
    """Handle weather-related queries"""
    log_execution(state, "weather_node", "STARTED")

    try:
        entities = state.get("entities", {})
        city = entities.get("city", "New Delhi")  # Default city

        # Call the tool directly with parameters
        result = search_weather.invoke({"usrdata": {"city": "Delhi", "unit": "Celsius"}})

        state["weather_data"] = result
        log_execution(state, "weather_node", "SUCCESS", f"Retrieved weather for {city}")

        return state

    except Exception as e:
        state["errors"].append(f"Weather search failed: {str(e)}")
        log_execution(state, "weather_node", "ERROR", str(e))
        state["retry_count"] = state.get("retry_count", 0) + 1
        return state

def news_node(state: ComplexAgentState) -> ComplexAgentState:
    """Handle news-related queries"""
    log_execution(state, "news_node", "STARTED")

    try:
        entities = state.get("entities", {})
        topic = entities.get("topic", "general")
        region = entities.get("city", "India")

        # Call the tool directly with parameters
        result = search_news.invoke({"topic": topic, "region": region})

        state["news_data"] = result
        log_execution(state, "news_node", "SUCCESS", f"Retrieved news for {topic} in {region}")

        return state

    except Exception as e:
        state["errors"].append(f"News search failed: {str(e)}")
        log_execution(state, "news_node", "ERROR", str(e))
        state["retry_count"] = state.get("retry_count", 0) + 1
        return state

def general_search_node(state: ComplexAgentState) -> ComplexAgentState:
    """Handle general search queries"""
    log_execution(state, "general_search_node", "STARTED")

    try:
        query = state["user_input"]

        # Call the tool directly with parameters
        result = search_web.invoke({"query": query, "num_results": 3})

        state["general_search_data"] = result
        log_execution(state, "general_search_node", "SUCCESS", f"Retrieved search results for: {query}")

        return state

    except Exception as e:
        state["errors"].append(f"General search failed: {str(e)}")
        log_execution(state, "general_search_node", "ERROR", str(e))
        state["retry_count"] = state.get("retry_count", 0) + 1
        return state

def error_handler_node(state: ComplexAgentState) -> ComplexAgentState:
    """Handle errors and decide on retries"""
    log_execution(state, "error_handler_node", "STARTED")

    retry_count = state.get("retry_count", 0)

    if retry_count < 2:  # Max 2 retries
        log_execution(state, "error_handler_node", "RETRY", f"Retry attempt {retry_count + 1}")
        state["next_action"] = "retry"
    else:
        log_execution(state, "error_handler_node", "GIVE_UP", "Max retries reached")
        state["next_action"] = "finalize"

    return state

def response_synthesizer(state: ComplexAgentState) -> ComplexAgentState:
    """Synthesize final response from all collected data"""
    log_execution(state, "response_synthesizer", "STARTED")

    try:
        response_parts = []

        # Add weather data if available
        if state.get("weather_data"):
            response_parts.append(f"🌤️ Weather Information:\n{state['weather_data']}")

        # Add news data if available
        if state.get("news_data"):
            response_parts.append(f"📰 News Information:\n{state['news_data']}")

        # Add general search data if available
        if state.get("general_search_data"):
            response_parts.append(f"🔍 Search Results:\n{state['general_search_data']}")

        # Handle errors
        if state.get("errors"):
            response_parts.append(f"⚠️ Errors encountered:\n" + "\n".join(state["errors"]))

        # Create final response
        if response_parts:
            final_response = "\n\n".join(response_parts)
        else:
            final_response = "I apologize, but I couldn't retrieve any information for your query."

        final_response = model.invoke([{"role": "assistant", "content": final_response},
                      {"role": "user", "content": "Organize this information into a concise pointwise response."},
                      ])

        state["final_response"] = final_response.content
        log_execution(state, "response_synthesizer", "SUCCESS", "Final response created")
        print(f"Final Response in synthesizer: {final_response}")
        return state

    except Exception as e:
        state["errors"].append(f"Response synthesis failed: {str(e)}")
        log_execution(state, "response_synthesizer", "ERROR", str(e))
        state["final_response"] = "An error occurred while processing your request."
        return state

# ===== ROUTING FUNCTIONS =====

def should_get_weather(state: ComplexAgentState) -> bool:
    """Check if we should get weather data"""
    return state.get("intent") in ["weather", "general"] and not state.get("weather_data")

def should_get_news(state: ComplexAgentState) -> bool:
    """Check if we should get news data"""
    return state.get("intent") in ["news", "general"] and not state.get("news_data")

def should_do_general_search(state: ComplexAgentState) -> bool:
    """Check if we should do general search"""
    return state.get("intent") == "search" and not state.get("general_search_data")

def should_handle_error(state: ComplexAgentState) -> bool:
    """Check if we should handle errors"""
    return len(state.get("errors", [])) > 0 and state.get("retry_count", 0) < 2

def route_after_classification(state: ComplexAgentState) -> str:
    """Route after intent classification"""
    if should_handle_error(state):
        return "handle_error"
    elif should_get_weather(state):
        return "weather"
    elif should_get_news(state):
        return "news"
    elif should_do_general_search(state):
        return "general_search"
    else:
        return "synthesizer"

def route_after_data_collection(state: ComplexAgentState) -> str:
    """Route after data collection nodes"""
    if should_handle_error(state):
        return "handle_error"
    elif should_get_weather(state):
        return "weather"
    elif should_get_news(state):
        return "news"
    elif should_do_general_search(state):
        return "general_search"
    else:
        return "synthesizer"

def route_after_error_handler(state: ComplexAgentState) -> str:
    """Route after error handling"""
    if state.get("next_action") == "retry":
        return route_after_classification(state)
    else:
        return "synthesizer"

# ===== CREATE COMPLEX WORKFLOW =====

def create_complex_workflow():
    """Create a complex workflow with multiple nodes and error handling"""

    workflow = StateGraph(ComplexAgentState)

    # Add all nodes
    workflow.add_node("classifier", intent_classifier)
    workflow.add_node("weather", weather_node)
    workflow.add_node("news", news_node)
    workflow.add_node("general_search", general_search_node)
    workflow.add_node("handle_error", error_handler_node)
    workflow.add_node("synthesizer", response_synthesizer)

    # Define edges
    workflow.add_edge(START, "classifier")

    # Conditional edges from classifier
    workflow.add_conditional_edges(
        "classifier",
        route_after_classification,
        {
            "weather": "weather",
            "news": "news",
            "general_search": "general_search",
            "handle_error": "handle_error",
            "synthesizer": "synthesizer"
        }
    )

    # Conditional edges from data collection nodes
    workflow.add_conditional_edges(
        "weather",
        route_after_data_collection,
        {
            "weather": "weather",
            "news": "news",
            "general_search": "general_search",
            "handle_error": "handle_error",
            "synthesizer": "synthesizer"
        }
    )

    workflow.add_conditional_edges(
        "news",
        route_after_data_collection,
        {
            "weather": "weather",
            "news": "news",
            "general_search": "general_search",
            "handle_error": "handle_error",
            "synthesizer": "synthesizer"
        }
    )

    workflow.add_conditional_edges(
        "general_search",
        route_after_data_collection,
        {
            "weather": "weather",
            "news": "news",
            "general_search": "general_search",
            "handle_error": "handle_error",
            "synthesizer": "synthesizer"
        }
    )

    # Error handler routing
    workflow.add_conditional_edges(
        "handle_error",
        route_after_error_handler,
        {
            "weather": "weather",
            "news": "news",
            "general_search": "general_search",
            "synthesizer": "synthesizer"
        }
    )

    # End at synthesizer
    workflow.add_edge("synthesizer", END)

    return workflow.compile()

# ===== TEST THE COMPLEX WORKFLOW =====

def test_complex_workflow():
    """Test the complex workflow with different queries"""

    # Create the workflow
    complex_agent = create_complex_workflow()

    # Test cases
    test_cases = [
        "What's the weather like in Delhi and give me latest technology news?",
        "Search for information about latest model of xai, grok and its information",
        "Give me news related to latest IPOs in india",
    ]

    for i, query in enumerate(test_cases, 1):
        print(f"\n{'🚀 TEST CASE ' + str(i) + ' ':=^60}")
        print(f"Query: {query}")

        # Create initial state
        initial_state = {
            "user_input": query,
            "intent": None,
            "entities": None,
            "weather_data": None,
            "news_data": None,
            "general_search_data": None,
            "errors": [],
            "retry_count": 0,
            "execution_log": [],
            "final_response": "",
            "next_action": None
        }

        # Run the workflow
        result = complex_agent.invoke(initial_state)

        # Show final result
        print(f"\n{'📋 FINAL RESULT ':=^60}")
        print(result["final_response"])

        # # Show execution log
        # print(f"\n{'📊 EXECUTION LOG ':=^60}")
        # for log_entry in result["execution_log"]:
        #     print(f"[{log_entry['timestamp']}] {log_entry['node']} - {log_entry['status']}")
        #     if log_entry['details']:
        #         print(f"   {log_entry['details']}")

        print(f"\n{'='*60}")

# Run the test
if __name__ == "__main__":
    test_complex_workflow()


=======================🚀 TEST CASE 1 =======================
Query: What's the weather like in Delhi and give me latest technology news?

🔄 [2025-07-12T18:13:40.956844] NODE: intent_classifier | STATUS: STARTED

🔄 [2025-07-12T18:13:40.956844] NODE: intent_classifier | STATUS: SUCCESS
   📝 Details: Intent: weather, Entities: {'city': 'Delhi', 'topic': 'technology'}

🔄 [2025-07-12T18:13:40.956844] NODE: weather_node | STATUS: STARTED
Received user data for weather search: city='Delhi' unit='Celsius' of type <class '__main__.WeatherSearchInput'>

🔄 [2025-07-12T18:13:43.451679] NODE: weather_node | STATUS: SUCCESS
   📝 Details: Retrieved weather for Delhi

🔄 [2025-07-12T18:13:43.452205] NODE: response_synthesizer | STATUS: STARTED

🔄 [2025-07-12T18:13:46.056624] NODE: response_synthesizer | STATUS: SUCCESS
   📝 Details: Final response created
Final Response in synthesizer: content="Okay, here's a concise summary of the weather information for Delhi:\n\n*   **Location:** Delhi, Ontario, Ca